In [12]:
import sys
from dotmap import DotMap

sys.path.append('../../')

from utils.io import load_yaml

server = 'local'
server = DotMap(load_yaml('../../config/servers.yml')[server])

In [13]:
import pandas as pd

meta_df_name = '../21_06_21_cholec80_dissection/cholec80_splits_transforms.pkl'
meta_df = pd.read_pickle(meta_df_name)
meta_df.aug_transforms = None  # no transforms for evaluation
meta_df.pre_transforms = meta_df.pre_transforms.transform(lambda x:
    [
        x[0],
        {
            'module': 'torchvision.transforms',
            'type': 'Resize',
            'kwargs': {'size': [480, 640]}
        },
        {
            'module': 'kornia.color',
            'type': 'RgbToBgr',
            'kwargs': {}
        }
    ]
)


# meta_df.pre_transforms.iloc[0].append(
#     {
#         'module': 'kornia.color',
#         'type': 'RgbToBgr',
#         'kwargs': {}
#     }
# )
# meta_df
meta_df.train = False
meta_df_sample = meta_df.sample(frac=0.1, random_state=42)
meta_df_sample.train = True
meta_df.update(meta_df_sample)
meta_df.to_pickle('cholec80_splits_transforms.pkl')
meta_df.iloc[0].pre_transforms

[{'module': 'utils.transforms',
  'type': 'Crop',
  'kwargs': {'top_left_corner': [58, 139],
   'shape': [421, 555],
   'order': 'chw'}},
 {'module': 'torchvision.transforms',
  'type': 'Resize',
  'kwargs': {'size': [480, 640]}},
 {'module': 'kornia.color', 'type': 'RgbToBgr', 'kwargs': {}}]

In [14]:
import os
import pandas as pd
from dotmap import DotMap

from utils.io import load_yaml
from utils.io import load_pickle, save_pickle
from lightning_data_modules import VideoDataModule

# shuffle meta df
meta_df = meta_df.sample(frac=0.1, random_state=1)

test_md_name = 'cholec80_fraction_shuffle_splits_test_md.pkl' # -> 'cholec80_fraction_shuffle_splits_test_md_full.pkl'
test_md = None
print('\nLooking for {}'.format(test_md_name))
if os.path.exists(test_md_name):
    test_md = load_pickle(test_md_name)
    print('Found.')
else:
    print('Could not find {}. Generating metadata.'.format(test_md_name))

# load cholec80 splits
prefix = server.database.location
clip_length_in_frames = 10
frames_between_clips = clip_length_in_frames
frame_rate = 5
train_split = 0.8
batch_size = 1
num_workers = 4
random_state = 42

dm = VideoDataModule(
    meta_df,
    prefix=prefix,
    clip_length_in_frames=clip_length_in_frames,
    frames_between_clips=frames_between_clips,
    frame_rate=frame_rate,
    train_split=train_split,
    batch_size=batch_size,
    num_workers=num_workers,
    random_state=random_state,
    test_metadata=test_md
)

_, _, test_md = dm.setup('test')

# store metadata
save_pickle(test_md_name, test_md)



Looking for cholec80_fraction_shuffle_splits_test_md.pkl
Found.


In [15]:
import cv2
import numpy as np
import os
from kornia import tensor_to_image
from tqdm import tqdm

from utils.io import generate_path

test_dl = dm.test_dataloader()

cnt = 0
max_cnt = 100
for batch in tqdm(test_dl):
    vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

    path = 'out/homography_labelling/vid_idx_{}/sample_idx_{}'.format(video_idx.item(), idx.item())
    generate_path(path)
    for frame_idx, frame in enumerate(vid[0]):

        

        frame = tensor_to_image(frame)

        cv2.imwrite(os.path.join(path, 'frame_idx_{}.png'.format(frame_idx)), (frame*255).astype(np.uint8))
        # cv2.imshow('i', frame)
        # cv2.waitKey()

    cnt += 1

    if cnt >= max_cnt:
        break

cv2.destroyAllWindows()

  0%|          | 0/480 [00:00<?, ?it/s]/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit